This notebook contains code on how to convert the isochrone polygons (and other component data) to arrays, which can be then processed parallely on GPU for stuff like weighted distance decay.

In [ ]:
import geopandas as gpd
import pandas as pd
from geocube.api.core import make_geocube
import numpy as np
import rasterio
import os
from multiprocess import Pool

In [ ]:
# set path for grid files
# you might have grids generated here from the 'generate_grids.ipynb'
grid_path = 'data/*.parquet'

# create a list of all parquet grid files from the specified directory
grids_list = [parquet for parquet in glob.glob(grid_path)]
print(grids_list)

# # configure logging (recommended if you monitor processing over a lot of files)
# log_path = 'logs/isochrones_to_gpu.log'

# # ensure log directory exists
# log_dir = os.path.dirname(log_path)
# if not os.path.exists(log_dir):
#     os.makedirs(log_dir)
    
# logging.basicConfig(filename=log_path, level=logging.INFO,
#                     format='%(asctime)s:%(levelname)s:%(message)s', force=True)

In [ ]:
def find_neighbors(grid_gdf, grid_100km_gdf, x_cols):
    """
    Find neighbors of a grid cell by buffering its boundaries and identifying intersecting grids.
    
    Args:
    - grid_gdf (GeoDataFrame): GeoDataFrame of the grid to find neighbors for.
    - grid_100km_gdf (GeoDataFrame): 100kmx100km GeoDataFrame consisting all grid ids.
    - x_cols (List): List of walkability components.
    
    Returns:
    - List[GeoDataFrame]: List of GeoDataFrames of the neighboring grid cells.
    """
    grid_id = grid_gdf['grid_100000_id'].unique()
    buffer = grid_gdf.unary_union.buffer(3000, cap_style=3)
    potential_neighbors = grid_100km_gdf[grid_100km_gdf.intersects(buffer)]
    
    nbr_gdfs = []
    for neighbor_id in set(potential_neighbors.index) - set(grid_id):
        temp_nbr_gdf = gpd.read_parquet(f'data/grids_100_{neighbor_id}.parquet')
        temp_nbr_gdf = temp_nbr_gdf[temp_nbr_gdf.intersects(buffer)]

        # check if any data col is missing
        # if missing we replace it by 0 (this is only limited to neighbors so shouldn't be an issue?)
        if not set(x_cols).issubset(set(temp_nbr_gdf.columns)):        
            missing_cols = list(set(x_cols) - set(temp_nbr_gdf.columns))
            nbr_id = temp_nbr_gdf['grid_100000_id'].unique()
            print(f'{missing_cols} missing for {grid_id} nbr - {nbr_id}, replacing with 0...')
            # logging.info(f'{missing_cols} missing for {grid_id} nbr - {nbr_id}, replacing with 0...')
            for missing_col in missing_cols:
                temp_nbr_gdf[missing_col] = 0.0
        
        nbr_gdfs.append(temp_nbr_gdf)
    
    return nbr_gdfs

def rasterize_geodf(geodf, columns, resolution):
    """
    Rasterizes specified columns of a GeoDataFrame.
    
    Args:
    - geodf (GeoDataFrame): GeoDataFrame to rasterize.
    - columns (List[str]): List of column names to rasterize.
    - resolution (tuple): The pixel resolution in the form of (width, height).
    
    Returns:
    - Tuple: A tuple containing the rasterized data array, transform, and CRS.
    """
    cube = make_geocube(vector_data=geodf, measurements=columns, resolution=resolution, output_crs="EPSG:3035")
    bands = [cube[col].values for col in columns]  # test if 'cube[col].data' is faster?
    return np.stack(bands), cube.rio.transform(), cube.rio.crs

def process_grid(grid_path, grid_100km_gdf, x_cols):
    """
    Process each grid to find neighbors, merge data, and create a raster image for GPU processing.
    
    Args:
    - grid_path (str): Path to the grid file.
    """
    grid_gdf = gpd.read_parquet(grid_path)
    grid_num = grid_gdf['grid_100000_id'].unique()[0]
    
    output_dir = 'data/iso_for_gpu'
    os.makedirs(output_dir, exist_ok=True)
    output_indexing_arr_file_path = os.path.join(output_dir, f'indexing_arr_{grid_num}.npz')
    output_img_file_path = os.path.join(output_dir, f'img_{grid_num}.tif')
    
    if os.path.exists(os.path.join(output_dir, output_img_file_path)):
        print(f'Skipping grid {grid_num} as iso_gpu already calculated')
        return
    
    # some component from main grid missing
    if not set(x_cols).issubset(set(grid_gdf.columns)):
        print(f'skipping grid {grid_num} as w_col missing: main_grid_id')
        # logging.info(f'skipping grid {grid_num} as w_col missing: main_grid_id')
        return
    
    nbr_gdfs = find_neighbors(grid_gdf=grid_gdf, grid_100km_gdf=grid_100km_gdf, x_cols=x_cols)
    new_grid_gdf = gpd.GeoDataFrame(pd.concat([grid_gdf] + nbr_gdfs, ignore_index=True))
    new_grid_gdf['retain_nbr_geom'] = new_grid_gdf.geometry
    new_grid_gdf.drop(columns=['index'], inplace=True)
    
    iso_path = f'data/isochrones/isochrones_{grid_num}.parquet'
    iso_gdf = gpd.read_parquet(iso_path)

    # wherever iso has empty polygon fill with normal buffer polygon
    # this leads to uncertainty? (as normal buffer would contain larger amounts of nbrs and
    # thus more costlier weighted reduction in kernel? however this is best to avoid outliers)
    empty_pols = iso_gdf[iso_gdf.geometry.is_empty].index
    condition = (grid_gdf.loc[empty_pols]['population'] > 0) | (grid_gdf.loc[empty_pols]['street_walk_length'] > 0)
    filtered_entries = grid_gdf.loc[empty_pols][condition]
    buffered_pols = filtered_entries.geometry.centroid.buffer(1200) 
    iso_gdf.loc[filtered_entries.index, 'geometry'] = buffered_pols

    iso_gdf.drop(columns=['index'], inplace=True)

    joined_gdf = iso_gdf.sjoin(new_grid_gdf, how='left', predicate='intersects')
    joined_gdf.reset_index(inplace=True)

    joined_gdf_x = joined_gdf[['index', 'index_right']].copy()
    joined_gdf_x.set_index('index', inplace=True)

    # convert to an array of size (num neighborhoods, indices of local intersecting neighbors)
    joined_gdf_x['group_num'] = joined_gdf_x.groupby(joined_gdf_x.index).cumcount()
    res_df = joined_gdf_x.pivot(columns='group_num', values='index_right')
    res_df.reset_index(inplace=True)
    res_df.fillna(value=np.nan, inplace=True)

    # save indexing arr
    indexing_arr = res_df.to_numpy()
    print(indexing_arr.shape)
    np.savez_compressed(output_indexing_arr_file_path, array=indexing_arr)

    new_grid_gdf.reset_index(inplace=True)

    # save image data to be processed (new_grid_gdf + index from grid_gdf)
    f_grid_bands, f_grid_transform, f_grid_crs = rasterize_geodf(new_grid_gdf, x_cols, resolution=(-100, 100))
    grid_bands, grid_transform, grid_crs = rasterize_geodf(grid_gdf, x_cols[-1], resolution=(-100, 100))

    # calculate offset for placeing grid_bands into f_grid_bands
    grid_xmin, grid_ymin, grid_xmax, grid_ymax = grid_gdf.total_bounds
    f_grid_xmin, f_grid_ymin, f_grid_xmax, f_grid_ymax = new_grid_gdf.total_bounds
    x_offset = int((grid_xmin - f_grid_xmin) / abs(f_grid_transform[0]))
    y_offset = int((f_grid_ymax - grid_ymax) / abs(f_grid_transform[4]))

    # extract the dimensions of the reasterized gdf
    height, width = f_grid_bands.shape[1], f_grid_bands.shape[2]
    channels = len(x_cols) + len([x_cols[-1]])

    # intialize an empty array to hold the composite bands
    composite_bands = np.full((channels, height, width), np.nan, dtype=np.float32)

    # fill composite bands with nbr_bands data
    composite_bands[:channels-1, :, :] = f_grid_bands

    # fill composite bands with grid_bands data
    composite_bands[-1, y_offset:y_offset+grid_bands.shape[1], x_offset:x_offset + grid_bands.shape[2]] = grid_bands
    print(composite_bands.shape)
    
    # create a profile for the raster file
    profile = {
        'driver': 'GTiff',
        'count': composite_bands.shape[0],
        'dtype': composite_bands.dtype,
        'width': width,
        'height': height,
        'crs': f_grid_crs.to_string(),
        'transform': f_grid_transform
    }
    
    # write the raster to a single TIFF file with multiple bands
    with rasterio.open(output_img_file_path, 'w', **profile) as dst:
        for i in range(composite_bands.shape[0]):
            dst.write(composite_bands[i], i + 1)
    
    print(f'processed {grid_num}')

In [ ]:
x_cols = ['green_area', 'street_walk_length', 'num_street_intersections', 'ndvi', 'ent_5',
          'slope', 'population', 'pub_trans_count', 'index']

# 100km grid_gdf
grid_100km = gpd.read_file('data/grid_100km_surf.gpkg')
grid_100km.index += 1
grid_100km.head()

In [ ]:
# sequential
for elem in grids_list:
    process_grid(elem, grid_100km_gdf=grid_100km, x_cols=x_cols)

# # parallel
# num_processes = 5

# with Pool(processes=num_processes) as pool:
#     pool.map(process_grid, grids_list)